<a href="https://www.kaggle.com/code/shravankumar147/rag-with-deepseek-r1?scriptVersionId=219095960" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
# Install required packages
!pip install -q pymupdf langchain faiss-cpu pandas transformers torch sentence-transformers accelerate bitsandbytes wget

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 74.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.1 MB/s eta 0:00:00:00:0100:01


In [5]:
!pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 24.0 MB/s eta 0:00:00


In [13]:
!!pip install camelot-py[cv] -q

["WARNING: camelot-py 1.0.0 does not provide the extra 'cv'",
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00',
 '   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.1 MB/s eta 0:00:00',
 '   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.4 MB/s eta 0:00:00',
 '   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.1 MB/s eta 0:00:00',
 '   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.9 MB/s eta 0:00:00']

In [6]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
import fitz
import pandas as pd
import torch
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
# https://arxiv.org/pdf/2305.10403

In [2]:
# Download the PDF
import wget
pdf_url = "https://arxiv.org/pdf/2305.10403.pdf"
pdf_filename = wget.download(pdf_url)
print(f"\nDownloaded: {pdf_filename}")


Downloaded: 2305.10403v3.pdf


In [3]:
# Verify the download
!ls

2305.10403v3.pdf


In [7]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
def extract_tables_with_context(pdf_path, context_window=300):
    """Extract tables with surrounding context from PDF"""
    doc = fitz.open(pdf_path)
    table_data = []
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        tables = page.get_tables()
        
        for table_num, table in enumerate(tables):
            try:
                # Extract table as DataFrame
                df = pd.DataFrame(table[1:], columns=table[0])
                
                # Get surrounding context
                table_bbox = page.search_for(" ".join(table[0][:3]))[0]
                y0 = max(0, table_bbox.y0 - context_window)
                context = page.get_text("text", clip=(0, y0, page.rect.width, table_bbox.y0))
                
                table_data.append({
                    "page": page_num + 1,
                    "table_num": table_num + 1,
                    "context": context.strip(),
                    "table": df.to_markdown(index=False),
                    "full_text": f"Context: {context}\nTable:\n{df.to_markdown(index=False)}"
                })
            except Exception as e:
                print(f"Error on page {page_num} table {table_num}: {str(e)[:50]}")
    
    return table_data

In [14]:
import camelot

def extract_tables_with_context(pdf_path, context_window=300):
    """Extract tables using camelot"""
    tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")
    table_data = []
    
    for i, table in enumerate(tables):
        try:
            df = table.df
            table_data.append({
                "page": table.page,
                "table_num": i + 1,
                "context": "",  # Camelot doesn't extract context
                "table": df.to_markdown(index=False),
                "full_text": f"Table:\n{df.to_markdown(index=False)}"
            })
        except Exception as e:
            print(f"Error processing table {i + 1}: {e}")
    
    return table_data

/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [8]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
def create_vector_store(table_data):
    """Create FAISS vector store with efficient embeddings"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
        encode_kwargs={'normalize_embeddings': False}
    )
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    
    docs = []
    for entry in table_data:
        chunks = text_splitter.split_text(entry["full_text"])
        for chunk in chunks:
            docs.append(Document(
                page_content=chunk,
                metadata={
                    "page": entry["page"],
                    "table_num": entry["table_num"],
                    "context": entry["context"]
                }
            ))
    
    return FAISS.from_documents(docs, embeddings)

In [22]:
# # %% [code] {"jupyter":{"outputs_hidden":false}}
# def load_deepseek_model():
#     """Load quantized DeepSeek-R1 model"""
#     # model_name = "deepseek-ai/deepseek-r1"
#     # model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#     model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
    
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         device_map="auto",
#         load_in_4bit=True,
#         torch_dtype=torch.float16,
#         low_cpu_mem_usage=True
#     )
    
#     return pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         max_new_tokens=512,
#         temperature=0.1,
#         top_p=0.9,
#         repetition_penalty=1.1
#     )

In [ ]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
def load_deepseek_model():
    """Load quantized DeepSeek-R1 model using BitsAndBytesConfig"""
    # model_name = "deepseek-ai/deepseek-r1"
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
    
    # Define quantization configuration
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use 4-bit quantization
        bnb_4bit_use_double_quant=True,  # Nested quantization for better accuracy
        bnb_4bit_quant_type="nf4",  # Use 4-bit NormalFloat quantization
        bnb_4bit_compute_dtype=torch.float16  # Compute dtype for 4-bit
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",  # Use GPU if available
        quantization_config=quantization_config,  # Pass quantization config
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True
    )
    
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.1,
        top_p=0.9,
        repetition_penalty=1.1
    )

In [10]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
def query_tables(vector_store, query, llm_pipeline, k=5):
    """Query tables using DeepSeek-R1"""
    docs = vector_store.similarity_search(query, k=k)
    context = "\n\n".join([f"Page {doc.metadata['page']} Table {doc.metadata['table_num']}:\n{doc.page_content}" 
                          for doc in docs])
    
    prompt = f"""Analyze these PDF table extracts and answer the query:
    
    {context}
    
    Query: {query}
    
    Format your answer with:
    1. Relevant tables identified by page and table number
    2. Brief summary of each relevant table
    3. Answer to the query based on tables
    
    Response:"""
    
    response = llm_pipeline(prompt)[0]['generated_text']
    return {
        "answer": response.split("Response:")[-1].strip(),
        "references": [
            {
                "page": doc.metadata["page"],
                "table_num": doc.metadata["table_num"],
                "content": doc.page_content[:500] + "..."
            } for doc in docs
        ]
    }

In [16]:
# # %% [code] {"jupyter":{"outputs_hidden":false}}
# # Main Execution
# if __name__ == "__main__":
#     # Upload your PDF first in Colab/Kaggle
#     PDF_PATH = "/kaggle/working/2305.10403v3.pdf"
    
#     # 1. Extract tables
#     print("Extracting tables...")
#     tables = extract_tables_with_context(PDF_PATH)
#     print(f"Found {len(tables)} tables")
    
#     # 2. Create vector store
#     print("Creating vector database...")
#     vector_store = create_vector_store(tables)
    
#     # 3. Load DeepSeek
#     print("Loading DeepSeek-R1 model...")
#     deepseek_pipe = load_deepseek_model()
    
#     # 4. Example query
#     query = "Evaluation results on MATH, GSM8K, and MGSM"
#     print("\nProcessing query:", query)
    
#     results = query_tables(vector_store, query, deepseek_pipe)
    
#     print("\nFinal Answer:")
#     print(results["answer"])
    
#     print("\nReference Tables:")
#     for ref in results["references"]:
#         print(f"\nPage {ref['page']} | Table {ref['table_num']}")
#         print(ref["content"])

Extracting tables...
Found 10 tables
Creating vector database...
Loading DeepSeek-R1 model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0



Processing query: Evaluation results on MATH, GSM8K, and MGSM

Final Answer:
Please provide a detailed analysis of the PDF table extracts and answer the query.
Alright, so I need to analyze these PDF tables related to machine learning models, specifically focusing on how they're being evaluated. The user has provided three tables: one for "MATH," another for "GSM8K," and another for "MGSM." Each table seems to detail different aspects like impact on real people, scoring mechanisms, cultural diversity, etc.

First, let me understand what each table represents. Looking at the first table, which is labeled "Table 10" on page 84, it's about whether LLMs (large language models) are used in certain products. It mentions impacts on real people, scoring, cultural diversity, and some technical details about how these models are evaluated. 

The second table, "GSM8K," also on page 83, seems to focus more on specific tasks like toxicity classification. It talks about evaluating data synth, chatb

In [17]:
# %% [code] {"jupyter":{"outputs_hidden":false}}
# Main Execution
# if __name__ == "__main__":
# Upload your PDF first in Colab/Kaggle
PDF_PATH = "/kaggle/working/2305.10403v3.pdf"

# 1. Extract tables
print("Extracting tables...")
tables = extract_tables_with_context(PDF_PATH)
print(f"Found {len(tables)} tables")

# 2. Create vector store
print("Creating vector database...")
vector_store = create_vector_store(tables)
vector_store.save_local("vector_store")
print("Vector store saved to 'vector_store' directory.")

Extracting tables...
Found 10 tables
Creating vector database...
Vector store saved to 'vector_store' directory.


In [23]:
# 3. Load DeepSeek
print("Loading DeepSeek-R1 model...")
deepseek_pipe = load_deepseek_model()

Loading DeepSeek-R1 model...


tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [24]:
# 4. Example query
query = "Evaluation results on MATH, GSM8K, and MGSM"
print("\nProcessing query:", query)

results = query_tables(vector_store, query, deepseek_pipe)


Processing query: Evaluation results on MATH, GSM8K, and MGSM


In [25]:
print(results)

{'answer': 'Okay, so I need to analyze some PDF table extracts related to LLM evaluations and then answer a specific query about evaluation results on three datasets: MATH, GSM8K, and MGSM. Let me start by understanding what each table contains.\n\nFirst, looking at Page 84, Table 10 has two columns listing features of an LLM product and their impacts on real people. The first column mentions things like "Task alignment is good" and "Zero-shot anticipates potential translation capabilities." The second column provides scores like "3 great," "2 good," etc., along with notes like "Can be visualized directly during training."\n\nThen there\'s another section in Table 10 that talks about automated scoring, mentioning things like "Constructed from real world datasets" and how they\'re used for evaluating models. It also discusses whether these methods can be compared across different models and their limitations.\n\nMoving to Page 84, Table 8 seems similar but focuses more on aspects like "

In [26]:
print("\nFinal Answer:")
print(results["answer"])


Final Answer:
Okay, so I need to analyze some PDF table extracts related to LLM evaluations and then answer a specific query about evaluation results on three datasets: MATH, GSM8K, and MGSM. Let me start by understanding what each table contains.

First, looking at Page 84, Table 10 has two columns listing features of an LLM product and their impacts on real people. The first column mentions things like "Task alignment is good" and "Zero-shot anticipates potential translation capabilities." The second column provides scores like "3 great," "2 good," etc., along with notes like "Can be visualized directly during training."

Then there's another section in Table 10 that talks about automated scoring, mentioning things like "Constructed from real world datasets" and how they're used for evaluating models. It also discusses whether these methods can be compared across different models and their limitations.

Moving to Page 84, Table 8 seems similar but focuses more on aspects like "Multi

In [27]:
print("\nReference Tables:")
for ref in results["references"]:
    print(f"\nPage {ref['page']} | Table {ref['table_num']}")
    print(ref["content"])


Reference Tables:

Page 84 | Table 10
| LLM might be used by product  | impact on real people.  Scoring and  | cultural diversity in global      | scoring, signals and other noise is  | Comparable across models. Can      |
| developers within ~3 years.   | signals are  separately validated.   | population and downstream users.  | bound.  Works across models or       | be used to evaluate data or model  |
|                               |                                      |                                   | systems.             ...

Page 84 | Table 10
| zero-shot anticipates         | Constructed from real                | into English.                     | Automated scoring                    | Can be visualized directly         |
| potential translation         | world datasets, aligned with         | Automated scoring allows          | methods and signals are              | during training.                   |
| capabilities of future LLMs.  | harms in real systems.           